<a href="https://colab.research.google.com/github/ginttone/multi_deeplearning/blob/master/Notescale_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 악보(타임시리즈)

## 데이터 전처리

In [1]:
note_seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

행렬로 만들어 주는데 y값도 x에 영향을 받기 때문에

x1 , x2, x3, ... xn y 이런 모양을 만들어준다 

```
['g8', 'e8', 'e4', 'f8']
['e8', 'e4', 'f8', 'd8']
['e4', 'f8', 'd8', 'd4']
```
시퀀스 : x1,x2,x3를 가지고 y를 예측한다

In [2]:
note_seq[0:4], note_seq[1:5], note_seq[2:6]

(['g8', 'e8', 'e4', 'f8'], ['e8', 'e4', 'f8', 'd8'], ['e4', 'f8', 'd8', 'd4'])

In [3]:
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

### for문 돌리기



참고) for 문 모를 때

종이에 격자 놓고

각각의 i(변수) 쓰고 subset 써서 이런식으로 연상한다
 
       0   ['g8', 'e8', 'e4', 'f8']
       1   ['e8', 'e4', 'f8', 'd8']
       2   ['e4', 'f8', 'd8', 'd4']
       3   ['f8', 'd8', 'd4', 'c8']
       4   ['d8', 'd4', 'c8', 'd8']              

[4,8,12,...] -> 행렬 

['g8', 'e8', 'e4', 'f8'] -> ['g8', 'e8', 'e4']['f8']

데이터 마지막 부분이 4개1묶음이 안되고 3개가 남아서 그전까지로만 짤라준것

In [4]:
len(note_seq), range(len(note_seq)-4)

(54, range(0, 50))

for문 돌리기 

값을 4개씩 점핑하며 그 값을 뽑아주는 것을 for문으로 뽑아낸다
 ```
 for i in range(len(note_seq)-4):
  subset =note_seq[i:i+4]
  print(subset)
  ```
  결과
  ```
  ['g8', 'e8', 'e4', 'f8']
  ['e8', 'e4', 'f8', 'd8']
  ['e4', 'f8', 'd8', 'd4']
  ['f8', 'd8', 'd4', 'c8']
  ['d8', 'd4', 'c8', 'd8']
  ['d4', 'c8', 'd8', 'e8']
  ['c8', 'd8', 'e8', 'f8']
  ['d8', 'e8', 'f8', 'g8']
          ....
```



code2idx[item] 를 list(순번으로 접근해서 값을 가져옴)로 만들기

(음)계명 만들기:머신에 넣어줘야하기 때문에 숫자로 바꿔준다

code2idx -> 딕셔너리 에 key 넣어주는데  value로 숫자화 시켜 list로 넣어준다

```
for i in range(len(note_seq)-4):
  subset =note_seq[i:i+4]
  print(subset)
  for item in subset:
    print(code2idx[item])
```
결과
```
['g8', 'e8', 'e4', 'f8']
11
9
2
10
['e8', 'e4', 'f8', 'd8']
9
2
10
8
...
```

In [5]:
dataset = list()

for i in range(len(note_seq)-4):
  subset = note_seq[i:i+4]
  items = list()
  # print(subset)
  for item in subset:
    items.append(code2idx[item])
  # print(items)
  dataset.append(items)

print(dataset)

[[11, 9, 2, 10], [9, 2, 10, 8], [2, 10, 8, 1], [10, 8, 1, 7], [8, 1, 7, 8], [1, 7, 8, 9], [7, 8, 9, 10], [8, 9, 10, 11], [9, 10, 11, 11], [10, 11, 11, 4], [11, 11, 4, 11], [11, 4, 11, 9], [4, 11, 9, 9], [11, 9, 9, 9], [9, 9, 9, 10], [9, 9, 10, 8], [9, 10, 8, 1], [10, 8, 1, 7], [8, 1, 7, 9], [1, 7, 9, 11], [7, 9, 11, 11], [9, 11, 11, 9], [11, 11, 9, 9], [11, 9, 9, 2], [9, 9, 2, 8], [9, 2, 8, 8], [2, 8, 8, 8], [8, 8, 8, 8], [8, 8, 8, 8], [8, 8, 8, 9], [8, 8, 9, 3], [8, 9, 3, 9], [9, 3, 9, 9], [3, 9, 9, 9], [9, 9, 9, 9], [9, 9, 9, 9], [9, 9, 9, 10], [9, 9, 10, 4], [9, 10, 4, 11], [10, 4, 11, 9], [4, 11, 9, 2], [11, 9, 2, 10], [9, 2, 10, 8], [2, 10, 8, 1], [10, 8, 1, 7], [8, 1, 7, 9], [1, 7, 9, 11], [7, 9, 11, 11], [9, 11, 11, 9], [11, 11, 9, 9]]


### 딥러닝에 넣어주려 모양 다듬기

In [6]:
import numpy as np

datasets = np.array(dataset)

In [7]:
x_train = datasets[:,0:3]
x_train.shape

(50, 3)

In [8]:
y_train = datasets[:,3]
y_train.shape

(50,)

### 정규화 mean max 스케일링

code2idx의 Len


In [9]:
len(code2idx)

14

In [10]:
x_train = x_train/13 #len(code2idx)
x_train[3]

array([0.76923077, 0.61538462, 0.07692308])

In [11]:
X_train = np.reshape(x_train, (-1, 3, 1)) # tensor
X_train.shape, X_train[2]

((50, 3, 1), array([[0.15384615],
        [0.76923077],
        [0.61538462]]))

In [12]:
np.unique(y_train)

array([ 1,  2,  3,  4,  7,  8,  9, 10, 11])

## Model create

In [13]:
import tensorflow as tf

In [14]:
model = tf.keras.models.Sequential()

In [15]:
model.add(tf.keras.Input(shape=(3,1))) # input layer
model.add(tf.keras.layers.LSTM(128)) # hidden layer
model.add(tf.keras.layers.Dense(13, activation='softmax')) # output layer

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

batch_size 

내부적으로 교육시킬 때 해당되는 학습속도를 빨리하기 위해 필요 

50/1(50) , 50/5(10) 속도늘리고싶을때는 해당 batch_size를 크게하면 됨 

In [16]:
hist = model.fit(X_train, y_train, epochs=500, batch_size=10)

Epoch 1/500
5/5 [==============================] - 2s 7ms/step - loss: 2.5611 - accuracy: 0.2600
Epoch 2/500
5/5 [==============================] - 0s 6ms/step - loss: 2.5224 - accuracy: 0.3400
Epoch 3/500
5/5 [==============================] - 0s 7ms/step - loss: 2.4857 - accuracy: 0.3400
Epoch 4/500
5/5 [==============================] - 0s 4ms/step - loss: 2.4425 - accuracy: 0.3400
Epoch 5/500
5/5 [==============================] - 0s 5ms/step - loss: 2.3945 - accuracy: 0.3400
Epoch 6/500
5/5 [==============================] - 0s 5ms/step - loss: 2.3323 - accuracy: 0.3400
Epoch 7/500
5/5 [==============================] - 0s 5ms/step - loss: 2.2560 - accuracy: 0.3400
Epoch 8/500
5/5 [==============================] - 0s 5ms/step - loss: 2.1687 - accuracy: 0.3400
Epoch 9/500
5/5 [==============================] - 0s 4ms/step - loss: 2.0767 - accuracy: 0.3400
Epoch 10/500
5/5 [==============================] - 0s 4ms/step - loss: 2.0210 - accuracy: 0.3400
Epoch 11/500
5/5 [===========

## Evaluation

정확도가 얼마나 높은지 체크

In [17]:
model.evaluate(X_train, y_train)

2/2 [==============================] - 0s 8ms/step - loss: 1.1845 - accuracy: 0.5400


[1.1845146417617798, 0.5400000214576721]

In [18]:
X_train[4:5]

array([[[0.61538462],
        [0.07692308],
        [0.53846154]]])

In [19]:
model.predict(X_train[0:1])

array([[4.4053531e-06, 7.1160402e-03, 3.8786512e-02, 9.9950954e-03,
        9.9263685e-03, 4.8943680e-06, 2.5389832e-06, 2.3996043e-01,
        5.3898573e-02, 1.6598853e-01, 4.4320348e-01, 3.1110469e-02,
        2.6355060e-06]], dtype=float32)

[[[0.61538462], [0.07692308], [0.53846154]]]

값을 넣어주는 모양

first = 0.61538462

second = 0.07692308

third = 0.53846154

[[[first], [second], [third]]]

In [20]:
first = 0.61538462

second = 0.07692308

third = 0.53846154

In [23]:
# pred = model.predict([[[first], [second], [third]]])
pred = model.predict(X_train[0:1])

In [24]:
np.argmax(pred)

10